In [ ]:
# Import functions for this notebook
from Surface_mining_screening import (
    calculate_area_per_pixel,
    load_vector_file,
    process_data,
    calculate_vegetation_loss,
    plot_possible_mining,
    plot_vegetationloss_mining,
)
import os

# Get a list of all shapefiles in the 'data' folder
shapefile_folder = "data"
shapefiles = [file for file in os.listdir(shapefile_folder) if file.endswith(".shp")]

# Iterate over the shapefiles
for shapefile in shapefiles:
    district_name = os.path.splitext(shapefile)[0]  # Get the shapefile name without the extension

    start_year = 2018
    end_year = 2022
    product = 's2'
    threshold = -0.1
    vector_file = os.path.join(shapefile_folder, shapefile)  # Create the full path to the shapefile
    
    try:
        gdf, geom = load_vector_file(vector_file)
        sentinel, wofs = process_data(gdf, geom, start_year, end_year, product=product)
        veg_loss_annual, veg_loss_all = calculate_vegetation_loss(sentinel, threshold=threshold, product=product)
        vegetation_loss_mining = plot_possible_mining(sentinel, veg_loss_all, wofs, product=product)
        plot_vegetationloss_mining(sentinel, veg_loss_annual, vegetation_loss_mining, product=product)
        from datacube.utils.cog import write_cog
        write_cog(vegetation_loss_mining, 
        fname=f"{district_name}.tif",
        overwrite=True)
        write_cog(vegetation_loss_mining, fname=f"{district_name}.tif", overwrite=True)
    except Exception as e:
        print(f"Error processing {district_name}: {str(e)}")
        continue